# Домашнее задание № 2. Мешок слов

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from razdel import tokenize
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore")

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [2]:
! unzip -o labeled.csv.zip
! rm -rf __MACOSX/

Archive:  labeled.csv.zip
  inflating: labeled.csv             
  inflating: __MACOSX/._labeled.csv  


In [3]:
data = pd.read_csv('labeled.csv')

stops = stopwords.words("russian")

In [4]:
y = data.toxic

X = data.drop(columns="toxic")

In [5]:
razdel_tokenize = lambda x: [item.text for item in list(tokenize(x))]

In [6]:
default_tokenizer = CountVectorizer()

razdel_tokenizer = CountVectorizer(stop_words=stops, tokenizer=razdel_tokenize)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle = True)

In [8]:
X_train_def_count = default_tokenizer.fit_transform(X_train.comment)
X_test_def_count = default_tokenizer.transform(X_test.comment)

In [9]:
X_train_razdel_count = razdel_tokenizer.fit_transform(X_train.comment)
X_test_razdel_count = razdel_tokenizer.transform(X_test.comment)

#### Classification

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score

In [11]:
def evaluate_model(model, **kwargs) -> None:

    model.fit(kwargs["X_train"], kwargs["y_train"])

    pred_def = model.predict(kwargs["X_test"])

    print(f'Confusion matrix for the model: \n{ confusion_matrix(kwargs["y_test"], pred_def) }')

    print("\n----------------------------\n")

    print(classification_report(kwargs["y_test"], pred_def))

    print(f'Overall f1-score is {f1_score(kwargs["y_test"], pred_def)}')

**Trying gradient boosting**

In [12]:
cls_grad = GradientBoostingClassifier()

In [13]:
evaluate_model(cls_grad, X_train = X_train_def_count, X_test = X_test_def_count, y_train = y_train, y_test = y_test)

Confusion matrix for the model: 
[[2789   78]
 [ 987  470]]

----------------------------

              precision    recall  f1-score   support

         0.0       0.74      0.97      0.84      2867
         1.0       0.86      0.32      0.47      1457

    accuracy                           0.75      4324
   macro avg       0.80      0.65      0.65      4324
weighted avg       0.78      0.75      0.71      4324

Overall f1-score is 0.4688279301745636


In [14]:
evaluate_model(cls_grad, X_train = X_train_razdel_count, X_test = X_test_razdel_count, y_train = y_train, y_test = y_test)

Confusion matrix for the model: 
[[2781   86]
 [1074  383]]

----------------------------

              precision    recall  f1-score   support

         0.0       0.72      0.97      0.83      2867
         1.0       0.82      0.26      0.40      1457

    accuracy                           0.73      4324
   macro avg       0.77      0.62      0.61      4324
weighted avg       0.75      0.73      0.68      4324

Overall f1-score is 0.39771547248182765


**Let's try random forest then**

In [15]:
cls_forest = RandomForestClassifier()

In [16]:
evaluate_model(cls_forest, X_train = X_train_def_count, X_test = X_test_def_count, y_train = y_train, y_test = y_test)

Confusion matrix for the model: 
[[2687  180]
 [ 790  667]]

----------------------------

              precision    recall  f1-score   support

         0.0       0.77      0.94      0.85      2867
         1.0       0.79      0.46      0.58      1457

    accuracy                           0.78      4324
   macro avg       0.78      0.70      0.71      4324
weighted avg       0.78      0.78      0.76      4324

Overall f1-score is 0.5789930555555556


In [17]:
evaluate_model(cls_forest, X_train = X_train_razdel_count, X_test = X_test_razdel_count, y_train = y_train, y_test = y_test)

Confusion matrix for the model: 
[[2790   77]
 [ 961  496]]

----------------------------

              precision    recall  f1-score   support

         0.0       0.74      0.97      0.84      2867
         1.0       0.87      0.34      0.49      1457

    accuracy                           0.76      4324
   macro avg       0.80      0.66      0.67      4324
weighted avg       0.78      0.76      0.72      4324

Overall f1-score is 0.4886699507389163


Похоже, результат в любом случае становится хуже, но, вероятно, это может служить не столько индикатором худшего качества разметки, сколько индикатором недостаточного качества эмбедингов или обобщающей способности модели.

In [18]:
default_tokenizer = TfidfVectorizer()

razdel_tokenizer = TfidfVectorizer(stop_words=stops, tokenizer=razdel_tokenize)

In [19]:
X_train_def_tfidf = default_tokenizer.fit_transform(X_train.comment)
X_test_def_tfidf = default_tokenizer.transform(X_test.comment)

In [20]:
X_train_razdel_tfidf = razdel_tokenizer.fit_transform(X_train.comment)
X_test_razdel_tfidf = razdel_tokenizer.transform(X_test.comment)

**Test gradient boosing with TfidfVectorizer**

In [21]:
evaluate_model(cls_grad, X_train = X_train_def_tfidf, X_test = X_test_def_tfidf, y_train = y_train, y_test = y_test)

Confusion matrix for the model: 
[[2784   83]
 [1003  454]]

----------------------------

              precision    recall  f1-score   support

         0.0       0.74      0.97      0.84      2867
         1.0       0.85      0.31      0.46      1457

    accuracy                           0.75      4324
   macro avg       0.79      0.64      0.65      4324
weighted avg       0.77      0.75      0.71      4324

Overall f1-score is 0.45536609829488467


In [22]:
evaluate_model(cls_grad, X_train = X_train_razdel_tfidf, X_test = X_test_razdel_tfidf, y_train = y_train, y_test = y_test)

Confusion matrix for the model: 
[[2787   80]
 [1095  362]]

----------------------------

              precision    recall  f1-score   support

         0.0       0.72      0.97      0.83      2867
         1.0       0.82      0.25      0.38      1457

    accuracy                           0.73      4324
   macro avg       0.77      0.61      0.60      4324
weighted avg       0.75      0.73      0.68      4324

Overall f1-score is 0.3812532912058978


**And with random forest again**

In [23]:
evaluate_model(cls_forest, X_train = X_train_def_count, X_test = X_test_def_count, y_train = y_train, y_test = y_test)

Confusion matrix for the model: 
[[2702  165]
 [ 782  675]]

----------------------------

              precision    recall  f1-score   support

         0.0       0.78      0.94      0.85      2867
         1.0       0.80      0.46      0.59      1457

    accuracy                           0.78      4324
   macro avg       0.79      0.70      0.72      4324
weighted avg       0.78      0.78      0.76      4324

Overall f1-score is 0.5877231171092729


In [24]:
evaluate_model(cls_forest, X_train = X_train_razdel_count, X_test = X_test_razdel_count, y_train = y_train, y_test = y_test)

Confusion matrix for the model: 
[[2798   69]
 [ 963  494]]

----------------------------

              precision    recall  f1-score   support

         0.0       0.74      0.98      0.84      2867
         1.0       0.88      0.34      0.49      1457

    accuracy                           0.76      4324
   macro avg       0.81      0.66      0.67      4324
weighted avg       0.79      0.76      0.72      4324

Overall f1-score is 0.48910891089108915


Хм, tf-idf лишь подтвердил, что с разделовской токенизацией, классифицирующая способность становится хуже...

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [100]:
tfidf_vect = TfidfVectorizer(tokenizer=razdel_tokenize, 
                            ngram_range=(1,3),
                            max_df = .05,
                            min_df = 1e-4, 
                            max_features=500,
                            stop_words=stops,
                            sublinear_tf=True)

In [101]:
count_vect = TfidfVectorizer(tokenizer=razdel_tokenize, 
                            ngram_range=(1,3),
                            max_df = .05,
                            min_df = 15,
                            max_features=500,
                            stop_words=stops)

In [102]:
X_train_tfidf = tfidf_vect.fit_transform(X_train.comment)
X_test_tfidf = tfidf_vect.transform(X_test.comment)

In [103]:
X_train_count = count_vect.fit_transform(X_train.comment)
X_test_count = count_vect.transform(X_test.comment)

In [110]:
from sklearn.naive_bayes import CategoricalNB

cls_forest = RandomForestClassifier(n_estimators=500, max_depth=25, min_samples_leaf=2, class_weight="balanced")

In [111]:
evaluate_model(cls_forest, X_train = X_train_count.toarray(), X_test = X_test_count.toarray(), y_train = y_train, y_test = y_test)

Confusion matrix for the model: 
[[1519 1348]
 [ 216 1241]]

----------------------------

              precision    recall  f1-score   support

         0.0       0.88      0.53      0.66      2867
         1.0       0.48      0.85      0.61      1457

    accuracy                           0.64      4324
   macro avg       0.68      0.69      0.64      4324
weighted avg       0.74      0.64      0.64      4324

Overall f1-score is 0.6134453781512604


In [112]:
evaluate_model(cls_forest, X_train = X_train_tfidf.toarray(), X_test = X_test_tfidf.toarray(), y_train = y_train, y_test = y_test)

Confusion matrix for the model: 
[[1508 1359]
 [ 209 1248]]

----------------------------

              precision    recall  f1-score   support

         0.0       0.88      0.53      0.66      2867
         1.0       0.48      0.86      0.61      1457

    accuracy                           0.64      4324
   macro avg       0.68      0.69      0.64      4324
weighted avg       0.74      0.64      0.64      4324

Overall f1-score is 0.6141732283464567


## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [25]:
# ваш код